- 案件：[リサイクル品を扱うネットショップのリサーチと商品登録業務をお手伝い頂けませんか？](https://crowdworks.jp/proposals/193638313#scroll_to_message)
- Notionページ：https://www.notion.so/5364556d13fe44d6afd582e0a49fb29a
- staff2@polyphony.tokyo
- Muzaiko20230606

# 動作検証

## ブラウザを起動しAmazonページを開く

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(1)

# Amazon商品ページを開く
# 固定入力の場合
url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771&dc&fs=true&ds=v1%3ARF4exvpb8fMooDbZYD7vieGaq36oHTXneHKiM1%2FeeLs&qid=1686742155&rnid=3901642051&ref=sr_nr_n_1'
# 標準入力の場合
# print('▼ 調査対象の Amazon URL を入力してください')
# url = input()
browser.get(url)

# Amazon のウィンドウハンドラの保持　→　なくても大丈夫っぽい
amazon_window = browser.current_window_handle
# print(amazon_window)

# 今後取得するAmazon・ヤフオク情報を格納するためのリスト準備
data_list = []

In [ ]:
# 【動作テスト用】
# （ブラウザが開かれている状態で）指定のページを開く
browser.get('★')

# 大元（Amazon）のウィンドウハンドラの保持
amazon_window = browser.current_window_handle
print(amazon_window)

## Amazonページから情報を取得する

In [ ]:
# 【動作テスト用】
# 2023/06/14：すべて OR 任意の番号のヤフオクのページを開く
for i in range(0, 10):
    yafuoku_item_list = browser.find_elements(By.XPATH, '//a[@class="__ylink"]')[i].get_attribute('data-url')
    print(yafuoku_item_list)
    browser.find_element(By.XPATH, '//a[@class="__ylink"]').click()

In [ ]:
# Amazon商品＋ヤフオク商品群の基点要素を取得する
# ● 2023/06/14：Amazonページ上の情報取得・判定は行わず、すべてのヤフオクページを開くのでよさそう
# 　結構いろいろと頑張ったけど、処理が重いせいかヤフオク商品ページが開かれないことが多い
# →　1商品ずつ開いて、情報取得・出品判定・記録・出品を繰り返す方法（上記セル）
# →　元の方法の問題解消。ヤフオク商品ページを開けないのは .click() が機能していなかったから。.send_keys(Keys.ENTER) とすることで解消できた

# Amazon商品＋ヤフオク商品群の基点要素を取得する
amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
# print(item_list)

i = 1

# 親要素内の情報を取得する
for amazon_elem in amazon_yafuoku_item_list:

    # print(i)
    i = i + 1
    if i > 10:
        sys.exit()
        
    # Amazon商品名
    amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
    print('Amazon商品名：', amazon_name)
    
    # Amazon購入実績
    # try:
    #     amazon_sold_num = amazon_elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
    #     # print(amazon_sold_num)
    # except:
    #     # print('▲ Amazon購入実績情報なし ▲')
    #     pass

    # Amazon価格
    try:
        amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
        # print('Amazon価格：', amazon_price)
    except:
        # print('▲ Amazon商品価格なし ▲')
        pass

    # print('　↓')

    # Amazon商品URL
    # amazon_url = amazon_elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
    # print('Amazon商品URL：', amazon_url)
    
    # ヤフオク商品群の基点要素を取得する
    yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')
    
    # ヤフオク商品群の親要素内の情報を取得する
    for yafuoku_elem in yafuoku_item_list:
        
        # ヤフオク商品名
        try:
            yafuoku_name = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
            print('ヤフオク商品名：', yafuoku_name)
        except:
            print('▲ 該当ヤフオク商品なし ▲')
        
        # ヤフオク現在価格
        try:
            yafuoku_price_now = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
            # print('ヤフオク現在価格：', yafuoku_price_now)
        except:
            # print('▲ ヤフオク現在価格なし ▲')
            pass

        # # ヤフオク即決価格
        # try:
        #     yafuoku_price_final = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "即決")]').text.replace('即決 : ￥', ''))
        #     # print(yafuoku_price_final)
        # except:
        #     print('▲ ヤフオク即決価格なし ▲')

        # # ヤフオク残り時間
        # try:
        #     yafuoku_item_left = yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "残り")]').text
        #     print(yafuoku_item_left)
        # except:
        #     print('▲ ヤフオク残り時間情報なし ▲')

        # 現在価格での利益額計算
        profit_now = amazon_price - yafuoku_price_now
        # print('A-Y参考現在価格差：', profit_now, '円')

        # 即決価格での利益額計算
        # profit_final = amazon_price - yafuoku_price_final
        # print('A-Y即決価格差：', profit_final)

        # 原罪価格での利益率計算
        profit_rate_now = profit_now / yafuoku_price_now
        # print('参考現在利益率：', round(profit_rate_now * 100, 1), '%')

        # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
        if profit_now > 0: # 利益額での判定
        # if profit_rate_now > 0.1: # 利益率での判定
            # ヤフオク商品ページを開く
            print('● ヤフオク商品ページを開きます ●')
            yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)
            
            # try:
            #     yafuoku_name = yafuoku_elem.find_element(By.XPATH, './/a').text
            #     print('ヤフオク商品名：', yafuoku_name)
            #     print('● ヤフオク商品ページを開きまuす ●')
            #     yafuoku_elem.find_element(By.XPATH, './/a').click()
            # except:
            #     print('▲ ヤフオク商品ページを開けませんでした ▲')                
        else:
            print('▲ 利益を見込めないためヤフオク商品ページを開きませんでした ▲')

        # print('')
    # print('-')
    # break

In [ ]:
# Amazon限定品かどうかを判定する
amazon_name = '【Amazon.co.jp 限定】イミダペプチド ソフトカプセル【21日分】63粒 日本予防医薬'

# パターンA
# if '【Amazon限定ブランド】' in amazon_name or '[Amazon限定ブランド]' in amazon_name or '【Amazon 限定ブランド】' in amazon_name:
#     print('◆◆◆ アマゾン限定品のためスキップしました ◆◆◆')
#     # continue
# elif '【Amazon.co.jp限定】' in amazon_name or '【Amazon.co.jp 限定】' in amazon_name:
#     print('◆◆◆ アマゾン限定品のためスキップしました ◆◆◆')
#     # continue
# elif '[Amazonブランド]' in amazon_name:
#     print('◆◆◆ アマゾン限定品のためスキップしました ◆◆◆')
#     # continue
# elif '【アマゾン限定】' in amazon_name:
#     print('◆◆◆ アマゾン限定品のためスキップしました ◆◆◆')
#     # continue
# else:
#     print('アマゾン限定品ではありません')
  
# パターンB
if '【Amazon限定ブランド】' in amazon_name \
    or '[Amazon限定ブランド]' in amazon_name \
    or '【Amazon 限定ブランド】' in amazon_name \
    or '【Amazon.co.jp限定】' in amazon_name \
    or '【Amazon.co.jp 限定】' in amazon_name \
    or '[Amazonブランド]' in amazon_name \
    or '【アマゾン限定】' in amazon_name:
    print('◆◆◆ アマゾン限定品のためスキップしました ◆◆◆')
else:
    print('アマゾン限定品ではありません')

# パターンC
# amazon_strs = [
#     '【Amazon限定ブランド】',
#     '【Amazon.co.jp限定】',
#     '【Amazon.co.jp 限定】',
#     '[Amazonブランド]',
#     '【アマゾン限定】'
# ]

# for str in amazon_strs:
#     if str in amazon_name:
#         print('◆◆◆ アマゾン限定品のためスキップしました ◆◆◆')
#         break
#     else:
#         print('アマゾン限定品ではありません')

## ヤフオクページから情報を取得する

In [ ]:
# 開いているすべてのウィンドウハンドラを取得（リスト形式）し、最後のウィンドウをアクティブにする
handle_array = browser.window_handles
print(handle_array)
browser.switch_to.window(handle_array[-1])

# ヤフオク商品ページのウィンドウハンドラの保持　→　不要っぽい
yafuoku_window = browser.current_window_handle
print(yafuoku_window)
print('最新のヤフオク商品ページをアクティブにしました')

In [ ]:
# 【動作テスト用】
# Amazonページから続けて行う場合はこのセルの実行は不要
'''
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import pandas as pd

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)

# ヤフオク商品ページを開く
# url = 'https://page.auctions.yahoo.co.jp/jp/auction/v1094580770' $ 固定入力の場合
print('▼ 対象URLを入力してください') # 標準入力の場合
url = input()
browser.get(url)

In [ ]:
# 【動作テスト用】
# ページを直接開く
# browser.get('https://page.auctions.yahoo.co.jp/jp/auction/h1089535961')

In [ ]:
# ヤフオク商品名
yafuoku_name = browser.find_element(By.XPATH, '//h1').text
print('ヤフオク商品名：', yafuoku_name)

In [ ]:
# ・出品者が「匿名配送」希望でないこと
# →　商品ページのタイトルに「匿名配送」アイコンがないことで判定
try:
    # 匿名配送だったらその時点で情報取得をやめて画面を閉じる
    privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
    print('▲ 匿名配送のため本商品の情報収集を終了します。ヤフオク商品ページのウィンドウハンドラの取得から再実行してください ▲')
    # browser.close()
except:
    privacy = '匿名配送なし'
    print('匿名配送：', privacy)

In [ ]:
# アクティブウィンドウ（ヤフオク商品ページ）のURLを取得する
yafuoku_url = browser.current_url
print('ヤフオク商品URL：', yafuoku_url)

In [ ]:
# ヤフオク現在価格
yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
print('ヤフオク現在価格：', yafuoku_price_now)

if '税込' in yafuoku_price_now:
    yafuoku_price_now_int = int(yafuoku_price_now.replace('税込', '／').replace('円）', '').split('／')[1])
else:
    yafuoku_price_now_int = int(yafuoku_price_now.split('円')[0])

print('ヤフオク現在価格(int)：', yafuoku_price_now_int)

In [ ]:
# ヤフオク即決価格
try:
    yafuoku_price_bin = browser.find_element(By.XPATH, '//dd[@class="Price__value Price__value--buyNow"]').text.replace(' ', '')
except:
    yafuoku_price_bin = 'なし'
finally:
    print('ヤフオク即決価格：', yafuoku_price_bin)

In [ ]:
# 商品の状態
state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
print('状態：', state)

In [ ]:
# 入札数
bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
print('入札数：', bidnum)

In [ ]:
# ・出品者の評価が97％以上
# →　商品ページの出品者欄の「○○%」で判定
seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
print('評価：', seller_rating)

seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))
print('評価(int)：', seller_rating_int)

# 評価が 97% 未満だったらその時点で情報取得をやめて画面を閉じる
if seller_rating_int < 97:
    print('▲ 匿名配送のため本商品の情報収集を終了します。次のヤフオク商品ページのウィンドウハンドラの取得から再実行してください ▲')
    # browser.close()

In [ ]:
# 評価数は多い方が望ましい
seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
print('評価数：', seller_rating_num)

In [ ]:
# 送料の「詳細」ポップアップから配送情報を取得する

# 送料の「詳細」をクリックする
# ● 「詳細」ポップアップを開く必要はないのかも
# →　ポップアップを開かないとエラーになる
browser.find_element(By.XPATH, '//a[@id="postageDetailCurrent"]').click()
browser.implicitly_wait(1)
# 配送先から「青森県」を選ぶ
browser.find_element(By.XPATH, '//option[contains(text(), "青森県")]').click()
browser.implicitly_wait(1)

postage_due = browser.find_element(By.XPATH, '//dt[contains(text(), "送料負担")]/following-sibling::dd').text
print('送料負担：', postage_due)

# 配送情報を取得する
try:
    shipping = browser.find_element(By.XPATH, '//dt[@class="BidModal__postageName"]').text
    print('配送方法(詳細)：', shipping)

    postage_1 = browser.find_element(By.XPATH, '//div[@class="BidModal__postagePrice"]').text
    print('送料(詳細)：', postage_1)

    postage_int = int(postage_1.replace('円（税込）', ''))
    print('送料(int)：', postage_int)
except:
    pass

# 「✕」をクリックしてポップアップを閉じる
browser.find_element(By.XPATH, '//div[@class="BidModal__box BidModal__box--postage js-modal-box"]//a[contains(text(), "閉じる")]').click()

postage_from = browser.find_element(By.XPATH, '//p[@class="Price__postageFrom"]').text
print(postage_from)

try:
    send_date = browser.find_element(By.XPATH, '//dt[contains(text(), "発送開始")]/following-sibling::dd').text
except:
    send_date = '情報なし'
finally:
    print('発送開始：', send_date)

try:
    postage_2 = browser.find_element(By.XPATH, '//span[@class="Price__postageValue"]').text
except:
    postage_2 = '情報なし'
finally:
    print('送料(地域込)：', postage_2)

In [ ]:
# 商品説明文を取得する
# この内容を基に配送業者情報、営業所止め情報、直接取引情報、着払い情報などを取りたい
yafuoku_info = browser.find_element(By.XPATH, '//div[@class="ProductExplanation__commentArea"]').text
print('商品説明：', yafuoku_info)

In [ ]:
'''
# 【動作テスト用】
# 正規表現での文字列検索

# Pythonで文字列を抽出（位置・文字数、正規表現） 　
# https://note.nkmk.me/python-str-extract/

# 【Python】文字列を検索・抽出する方法【in演算子、findメソッド、正規表現】
# https://python-academia.com/python-str-extract/

import re

s = '直接取引限定　直接取り引き限定　直接取引のみ　直接引き取りのみ　要直接引き渡し　直接取引可能　営業所止め限定　営業所止め可能　営業所止め不可　着払い　着払い希望　着払い可能'

print(re.findall('直接.?.?.?.?.?.?', s))
print(re.findall('営業所.?.?.?.?', s))
print(re.findall('着払い.?.?.?.?', s))

print('直接取引限定：', '直接取引限定' in s)
print('営業所止め：', '営業所止め' in s)
print('着払い：', '着払い' in s)

In [ ]:
# ・直接引き取り限定でないこと
# →　商品タイトルおよび商品説明文に「直接取引のみ」などの単語がないことで判定
# →　▲ テキストマイニングを活用するとよい？？？
# →　いったん大丈夫そう

# 商品名判定
print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))

# 商品説明判定
print('直接引取(商品説明)：', re.findall('.*直接.*引.*', yafuoku_info))
print('手渡し(商品説明)', re.findall('.*手渡.*', yafuoku_info))

# print('直接取引(商品説明)：', '直接取引' in yafuoku_info)

In [ ]:
# ・配送が着払いでないこと
# →　「送料負担」が「出品者」であること OR 配送情報詳細や商品説明に「着払い」とないこと

print('送料負担：', postage_due) # 上で取得済

# 商品名判定
print('着払(商品名)：', re.findall('.*着払.*', yafuoku_name))

# 商品説明判定
print('着払(商品説明)：', re.findall('.*着払.*', yafuoku_info))

In [ ]:
# ・配送がヤマト、佐川、日本郵便などであること（直接エンドユーザーまで届けられること、一部運送会社だと営業所留めがあるため）
# ・営業所止めでないこと
# →　商品説明文に「営業所度止め」などの単語がないことで判定。「らくらく家財便」だと安心

# 商品名判定
print('ヤマト(商品名)：', re.findall('.*ヤマト.*', yafuoku_name))

# 商品説明判定
print('ヤマト(商品説明)：', re.findall('.*ヤマト.*', yafuoku_info))
print('らくらく家財便(商品説明)：', re.findall('.*らくらく.*', yafuoku_info))
print('佐川(商品説明)', re.findall('.*佐川.*', yafuoku_info))
print('日本郵便(商品説明)', re.findall('.*日本郵便.*', yafuoku_info))
print('日本郵便(商品説明)', re.findall('.*ゆう.*', yafuoku_info))
print('営業所止め(商品説明)', re.findall('.*止め.*', yafuoku_info))

In [ ]:
# 訳あり品でないこと（下遠野追加）

# 商品名判定
print('訳あり(商品名)：', re.findall('.*訳.*', yafuoku_name))

# 商品説明判定
print('訳あり(商品説明)：', re.findall('.*訳.*', yafuoku_info))

In [ ]:
# オークションの残り日数が一定数あること
# →　商品ページ内の残り時間やオークション終了用低日時で判定

auc_timeleft = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[1].text
print('残り日数(画面表示)：', auc_timeleft)

auc_enddatetime = browser.find_element(By.XPATH, '//th[contains(text(), "終了日時")]/following-sibling::td').text
print('終了予定(str型)：', auc_enddatetime)

auc_enddate = auc_enddatetime.split('（')[0]
auc_endtime = auc_enddatetime.split('）')[1]
auc_end = auc_enddate + ' ' + auc_endtime
auc_enddatetime = datetime.datetime.strptime(auc_end, '%Y.%m.%d %H:%M')
print('終了予定(datetime型)：', auc_enddatetime)
# print(type(auc_enddatetime))

dt_now = datetime.datetime.now()
print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))

auc_time_left = auc_enddatetime - dt_now
print('残り時間(詳細計算)：', auc_time_left)

In [ ]:
# アマゾンでの販売価格が適切に収まること
# →　新品価格を下回っていることで判定
profit = amazon_price - yafuoku_price_now_int - postage_int
print('A-Y現在利益(ヤフオク送料込)：', profit, '円　※Amazon出品手数料は含まれていません')

In [ ]:
# ヤフオクとの価格差がある程度あること（利益が取れること）
# →　ResaleTrapで判定するため本プログラムでは対応不要
# →　▲ プログラム上でもできそう
# →　Amazon 出品手数料が分からない
profit_rate = round(profit / yafuoku_price_now_int * 100, 1)
print('現在利益率(ヤフオク送料込)：', profit_rate, '%　※Amazon出品手数料は含まれていません')

In [ ]:
# 目視確認用の情報一覧表示
print('Amazon商品名：', amazon_name)
# print('Amazon商品URL：', amazon_url)
print(amazon_sold_num)
print('Amazon価格：', amazon_price)
print('ヤフオク商品名：', yafuoku_name)
# print('ヤフオク商品URL：', yafuoku_url)
print('ヤフオク現在価格：', yafuoku_price_now_int)
print('送料(詳細)：', postage_int)
print('送料(地域込)：', postage_2)
print('A-Y現在利益(ヤフオク送料込)：', profit, '※Amazon出品手数料は含まれていません')
print('現在利益率(ヤフオク送料込)：', round(profit_rate, 1), '%　※Amazon出品手数料は含まれていません')
print('状態：', state)
print('入札数：', bidnum)
print('評価：', seller_rating)
print('匿名配送：', privacy)
print('配送方法(詳細)：', shipping)
# print('送料(詳細)：', postage_1)
# print('送料(int)：', postage_int)
print('発送開始：', send_date)
# print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))
print('直接引取(商品説明)：', re.findall('.*直接.*引.*', yafuoku_info))
print('手渡し(商品説明)', re.findall('.*手渡.*', yafuoku_info))
print('送料負担：', postage_due)
# print('着払(商品名)：', re.findall('.*着払.*', yafuoku_name))
print('着払(商品説明)：', re.findall('.*着払.*', yafuoku_info))
print('ヤマト(商品説明)：', re.findall('.*ヤマト.*', yafuoku_info))
print('らくらく家財便(商品説明)：', re.findall('.*らくらく.*', yafuoku_info))
print('佐川(商品説明)', re.findall('.*佐川.*', yafuoku_info))
print('日本郵便(商品説明)', re.findall('.*ゆう.*', yafuoku_info))
print('営業所止め(商品説明)', re.findall('.*止め.*', yafuoku_info))
print('訳あり(商品名)：', re.findall('.*訳.*', yafuoku_name))
print('訳あり(商品説明)：', re.findall('.*訳.*', yafuoku_info))
print('終了予定(datetime型)：', auc_enddatetime)
print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))
print('残り時間(詳細計算)：', auc_time_left)
print('----- 商品説明 ----------------------------------------------------')
print(yafuoku_info)

In [ ]:
# 出品可否を判定する
judge_flag = 'OK'

# これ以降でNG判定だけを行えば良い。NGになったらフラグを 0 にして、最終的な値で判定する

# 出品者の評価が97％以上
if seller_rating_int >= 97:
    print('◯ 評価')
else:
    print('✕ 評価')
    judge_flag = 'NG'

# 出品者が「匿名配送」希望でないこと
if privacy != '匿名配送':
    print('◯ 匿名配送')
else:
    print('✕ 匿名配送')
    judge_flag = 'NG'
    
# 訳あり品でないこと
if len(re.findall('.*訳.*', yafuoku_name)) == 0 and len(re.findall('.*訳.*', yafuoku_info)) == 0:
    print('◯ 訳あり判定')
else:
    print('✕ 訳あり判定')
    judge_flag = 'NG'

# 直接引き取り限定でないこと
if len(re.findall('.*直接.*引.*', yafuoku_name)) == 0 and len(re.findall('.*直接.*引.*', yafuoku_info)) == 0 and len(re.findall('手渡.{10}', yafuoku_info)) == 0:
    print('◯ 直接引取判定')
else:
    print('✕ 訳あり判定')
    judge_flag = 'NG'

# 配送がヤマト、佐川、日本郵便などであること（直接エンドユーザーまで届けられること、一部運送会社だと営業所留めがあるため）
if len(re.findall('.*止め.*', yafuoku_info)) == 0:
    print('◯ 営業所止め判定')
else:
    print('✕ 営業所止め判定')
    judge_flag = 'NG'
    
# 配送が着払いでないこと
if len(re.findall('着払.*', yafuoku_name)) == 0 and len(re.findall('着払.*', yafuoku_info)) == 0:
    print('◯ 着払い判定')
else:
    print('✕ 着払い判定')
    judge_flag = 'NG'

print('')
print('↑　出品判定：', judge_flag)
print('↓　以下の条件も判定してください')
print('')

# アマゾンでの販売価格が適切に収まること　→　Amazonページで開く時に判定済
# ヤフオクとの価格差がある程度あること（利益が取れること）　→　Amazonページで開く時に判定済
print('Amazon価格：', amazon_price, '円')
print('ヤフオク現在価格：', yafuoku_price_now_int, '円')
print('ヤフオク送料：', postage_int, '円')
print('現在利益(ヤフオク送料込)：', profit, '円　※Amazon出品手数料は含まれていません')
print('現在利益率(ヤフオク送料込)：', round(profit_rate, 1), '%')

# オークションの残り日数が一定数あること
print('残り時間：', round(auc_time_left / datetime.timedelta(days=1), 1), '日')

In [ ]:
# 出品（監視対象に追加する）に値するかを判定し、合格の場合は出品する
# とりあえずは目視チェックにしてみる

print('この商品を出品しますか？( Y / N )')

while True:
    sell_flag = input()

    if sell_flag == 'Y':
        # 「監視対象に追加する」をクリックする。新しいウィンドウでヤフオクの商品ページが開かれる
        browser.find_element(By.XPATH, '//a[contains(text(), "監視対象に追加する")]').click()
        print('出品画面に進みます')
        break
    elif sell_flag == 'N':
        print('出品を取りやめます。ヤフオク商品ページのウィンドウハンドラの取得から再実行してください')
        # browser.close()
        break
    else:
        print('Y か N を入力してください')

In [ ]:
# 開いているすべてのウィンドウハンドルを取得（リスト形式）し、新しいウィンドウ（ResaleTrap管理画面）をアクティブにする
handle_array = browser.window_handles
# print(handle_array)
browser.switch_to.window(handle_array[-1])

# ResaleTrapページのウィンドウハンドラの保持　→　なくても大丈夫っぽい
resaletrap_window = browser.current_window_handle
# print(resaletrap_window)
print('ResaleTrap のページをアクティブにしました')

In [ ]:
# 出品情報を入力
yafuoku_price_bin = 10000
postage_int = 1000

# 予想落札価格　即決価格があれば入力
if yafuoku_price_bin != 'なし':
    browser.find_elements(By.XPATH, '//input')[0].send_keys(Keys.CONTROL,"a")
    browser.find_elements(By.XPATH, '//input')[0].send_keys(Keys.DELETE)
    browser.find_elements(By.XPATH, '//input')[0].send_keys(yafuoku_price_bin)

# 送料
browser.find_elements(By.XPATH, '//input')[1].send_keys(Keys.CONTROL,"a")
browser.find_elements(By.XPATH, '//input')[1].send_keys(Keys.DELETE)
browser.find_elements(By.XPATH, '//input')[1].send_keys(postage_int)

# 利益額　※ 利益「率」とのどちらか一方を選択
# browser.find_elements(By.XPATH, '//input')[2].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[2].send_keys('0')

# 利益率　※ 利益「額」とのどちらか一方を選択
# browser.find_elements(By.XPATH, '//input')[3].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[3].send_keys('0')

# リードタイム
# browser.find_elements(By.XPATH, '//input')[4].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[4].send_keys('0')

# 販売価格
# browser.find_elements(By.XPATH, '//input')[5].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[5].send_keys('0')

# Sub Condition
# ■ これを選択した後に別の項目に移るとデフォルト値に戻ってしまう
# browser.find_element(By.XPATH, '//option[@value="24"]').click()

# 商品説明　直接入力（ヤフオク商品情報転記）
# browser.find_elements(By.XPATH, '//textarea').send_keys(Keys.CONTROL,"a") # エラー「AttributeError: 'list' object has no attribute 'send_keys'」
# browser.find_elements(By.XPATH, '//textarea').send_keys(yafuoku_info)

# 商品説明　テンプレート選択
browser.find_element(By.XPATH, '//option[@value="6483"]').click()

In [ ]:
# Amazon 出品手数料を取得する
# 出品にかかる費用
# https://sell.amazon.co.jp/pricing?ld=SEJPSOAGoog__cmp-16999200710_adg-139400180041_ad-594040917490_kwd-363447672221_dev-c_ext-_prd-_sig-Cj0KCQjwqNqkBhDlARIsAFaxvwwZgcUGlSIzHVxD_a1PleJ3Vbc2ba2vW45QT1FnfgGkYkIy_d0uA18aAoiuEALw_wcB_asret
# →　ロジックが複雑そうなのでいったん保留

# amazon_charge = browser.find_elements(By.XPATH, '//★').text
# print(amazon_charge)

In [ ]:
# 多重ループを抜けるテスト
list_a = [0, 1, 2, 3, 4]
list_b = [5, 6, 7, 8, 9]

for i in list_a:
    print('run outer loop')
    for j in list_b:
        print('run inner loop')
        print(i, j)
        if i == 1 and j == 6:
            print('break inner loop')
            break
    else:
        print('continue')
        continue
    print('break outer loop')
    break

In [ ]:
# 次のページへ移る

url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771&dc&fs=true&ds=v1%3A%2F4aS563Nkvk0UGsPleVbnbegcIvY1CBz%2B04FQKejLmc&qid=1686834426&ref=sr_ex_n_1'

while True:
    browser.get(url)
    
    print('=== 情報取得処理 ===')
    
    print('処理をやめますか？(yes / no)')
    s = input()
    if s == 'yes':
        sys.exit()
    else:
        url = browser.find_element(By.XPATH, '//a[contains(text(), "次へ")]').get_attribute('href')

In [ ]:
# 取得した情報を CSV 出力用にまとめる（出品判定の証跡として残す）
get_datetime = datetime.datetime.now().strftime('%Y/%m/%d %H:%M')

data = {
    '出品判定': judge_flag,
    'Amazon商品名': amazon_name,
    'Amazon価格': amazon_price,
    # 'Amazon出品手数料': amazon_charge,
    'Amazon商品URL': amazon_url,
    'ヤフオク商品名': yafuoku_name,
    'ヤフオク現在価格': yafuoku_price_now,
    'ヤフオク商品URL': yafuoku_url,
    'A-Y現在利益(送料込)': profit,
    '現在利益率(送料込)': profit_rate,
    '情報取得日時': get_datetime,
    }

data_list.append(data)
print('data_list', data_list)

## ResaleTrap で出品する

In [ ]:
# 「新規出品」ボタンをクリックする
# まずは人力でスタートし、一定の実績が積めてから自動化する
# ■ 「取得したAmazon情報・ヤフオク情報を蓄積する」と1セットにすることで履歴残し漏れ・出品漏れを防げる

browser.find_element(By.XPATH, '//button[contains(text(), "新規出品")]').click()

In [ ]:
# ResaleTrap ページを閉じる
browser.close()
print('ResaleTrap のページを閉じました')

In [ ]:
# 出品が完了したヤフオク商品ページも閉じる
handle_array = browser.window_handles
# print(handle_array)
browser.switch_to.window(handle_array[-1])

browser.close()
print('出品が完了したヤフオク商品ページを閉じました')
print('次のヤフオク商品ページをアクティブにするところから再実行してください')

# 次のヤフオク商品ページをアクティブにする
# →　「ヤフオクページから情報を取得する」項に戻る

## 蓄積したAmazon情報・ヤフオク情報をCSV出力する

In [ ]:
# 取得した情報を CSV ファイルに出力する
output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')

## 構想版　Amazonページとヤフオクページをそれぞれ入力して調査する
1. Amazonページ＋ResaleTrapで目視で同一商品を探す
1. 該当のヤフオクページを開く
1. 本プログラムを起動する
1. AmazonページとヤフオクページのURLを入力する
1. 両サイトから必要情報を取得・調査する
1. 条件を満たしていればResaleTrapでの出品画面に移る
1. 手動で出品する
1. (プログラムにて出品情報を出力する)
- Amazon関連の変数は「A_」、ヤフオク関連の変数は「Y_」とする

In [ ]:
# モジュールのインポート

# ブラウザ設定

# データ格納用リスト準備
data_list = []

# Amazon情報を取得
print('調査対象のAmazon URLを入力してください')
A_url = input()
browser.get(A_url)


# ヤフオク情報を取得
print('調査対象のヤフオクURLを入力してください')
Y_url = input()

# Amazon情報とヤフオク情報から出品判定

# ResaleTrapで出品

# Amazon情報とヤフオク情報を出力


# 完成版　関数化パターン

In [ ]:
import xxx

def aaa():
    pass

def bbb():
    pass

# csvファイルに出力する
def output():
    pass

def main():
    aaa()
    bbb()
    output()

if __name__ == '__main__':
    main()

# 完成版　モノリシックパターン
- staff2@polyphony.tokyo
- Muzaiko20230606
- 2023/06/17 MTG
  - amazon 限定商品は除外する → 完了
  - 試しに10商品を出品して連絡する → 完了
  - 売れ筋商品情報をいただける

In [ ]:
''' 2023/07/17、完成版バックアップ
# 2023/06/18：Amazon商品名に「Amazon限定」関連の文言がある場合はスキップ
# 2023/06/23：Amazon商品名に「アイリスオーヤマ」がある場合はスキップ
# 2023/06/23：ヤフオクぉく商品名に「ジャンク」とあったらスキップ
# 2023/06/24：全3つのwhileループ判定を効率化してインデントを簡素化
# 2023/06/24：特定のAmazon商品に対するヤフオク商品の情報取得を一括スキップする方法（多重ループからのbreak：選択肢「S」）を追加
# 2023/06/24：ヤフオクぉく商品名に「訳あり」とあったらスキップ
# 2023/06/25：スポンサー商品をスキップ
# 2023/06/26：最終ページかどうかの判定を追加
# 2023/06/28：ResaleTrap ページで即決価格入力、送料入力、商品説明テンプレート選択を自動化
# 2023/06/28：次ページへの移動を簡素化
# 2023/06/30：ヤフオクの送料構造が「詳細」ポップアップから「配送方法一覧」に変わったことに対応
# 2023/06/30：現在価格と即決価格の取り方 XPATH を改良して即決価格だけの場合にも正しく取れるようにした

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# このプログラムで取得する Amazon・ヤフオク情報を格納するためのリスト準備
data_list = []

# 格納した情報の出力先ファイル名
output_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'

# 調査対象のAmazon商品ページを開く（固定入力パターン）
# url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771&dc&fs=true&ds=v1%3A%2F4aS563Nkvk0UGsPleVbnbegcIvY1CBz%2B04FQKejLmc&qid=1686834426&ref=sr_ex_n_1' # キッチン家電
# url = 'https://www.amazon.co.jp/s?k=%E3%82%A2%E3%83%9E%E3%82%BE%E3%83%B3+%E9%99%90%E5%AE%9A&__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&crid=3HVT9UVZAI6YQ&sprefix=%E3%82%A2%E3%83%9E%E3%82%BE%E3%83%B3+%E9%99%90%E5%AE%9A%2Caps%2C196&ref=nb_sb_noss_1' # アマゾン限定品

# 調査対象のAmazon商品ページを開く（手動入力パターン）
print('■ 調査対象の Amazon URL を入力してください')
url = input()

def main(url):
    # 2023/06/18：次のページに移動する処理を追加
    while True:
        browser.get(url)
        browser.implicitly_wait(3)
        # Amazon のウィンドウハンドラの保持
        amazon_window = browser.current_window_handle

        print('※ ResaleTrapの検索結果の整合性が低い場合は別のAmazonページに切り替えることをオススメします')
        print('')
        
        # 途中でエラーになった場合に備えて任意の位置(商品)から再開できるようする
        amazon_num_current = 1 # Amazon商品の現在番号
        print('■ ResaleTrapにログインしていなければログインしてください')
        print('staff2@polyphony.tokyo')
        print('Muzaiko20230606')
        print('')
        print('■ 何番目のAmazon商品から情報取得しますか？ 1～29　※スポンサー商品/Amazon限定品は自動スキップします')
        while True:
            amazon_num_temp = input() # 情報取得開始番号
            if re.fullmatch('[0-2]?[0-9]', amazon_num_temp):
                amazon_num_start = int(amazon_num_temp)
                break
            else:
                print('1～29の半角数字を入力してください')

        # Amazon商品＋ヤフオク商品群の基点要素を取得する
        amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

        # Amazon＋ヤフオク商品群の親要素内の情報を取得する
        for amazon_elem in amazon_yafuoku_item_list:

            # 途中でエラーになった場合に備えた仕組み。Amazon商品の情報取得を上で指定した番号までスキップする
            if amazon_num_current < amazon_num_start:
                amazon_num_current += 1
                continue

            print('')
            print('■■■■■■■■■■■■■■■ Amazon商品', amazon_num_current, '点目 ■■■■■■■■■■■■■■■')
            amazon_num_current += 1

            # Amazon商品名
            amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
            print(amazon_name)

            # Amazon限定商品はスキップする
            if '【Amazon限定ブランド】' in amazon_name \
              or '[Amazon限定ブランド]' in amazon_name \
              or '【Amazon 限定ブランド】' in amazon_name \
              or '【Amazon.co.jp限定】' in amazon_name \
              or '【Amazon.co.jp 限定】' in amazon_name \
              or '[Amazonブランド]' in amazon_name \
              or '【アマゾン限定】' in amazon_name:
                print('◆◆◆ Amazon限定品のためスキップしました ◆◆◆')
                continue
            # アイリスオーヤマ製品は注意 or スキップする
            elif 'アイリスオーヤマ' in amazon_name:
                print('◆◆◆ アイリスオーヤマ製品です。テレビの場合はスキップしてください ◆◆◆')

            # スポンサー商品はスキップする
            try:
                if amazon_elem.find_element(By.XPATH, './/span[contains(text(), "スポンサー")]').text == 'スポンサー':
                    print('◆◆◆ スポンサー商品のためスキップしました ◆◆◆')
                    continue
            except:
                pass

            # Amazon購入実績
            # try:
            #     amazon_sold_num = amazon_elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
            # except:
            #     amazon_sold_num = '◆Amazon購入実績情報なし◆'
            # finally:
                # print('Amazon購入実績：', amazon_sold_num)

            # Amazon価格
            try:
                amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
                print(amazon_price)
            except:
                amazon_price = '◆Amazon価格情報なし◆'
                print(amazon_price)
                continue

            # Amazon商品URL
            amazon_url = amazon_elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
            print(amazon_url)

            # ヤフオク商品群の基点要素を取得する
            yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')
            yafuoku_num_current = 1 # 同一Amazon商品内のヤフオク商品の現在番号

            # ヤフオク商品群の親要素内の情報を取得する
            for yafuoku_elem in yafuoku_item_list:
                # ヤフオク商品名
                try:
                    yafuoku_name_a = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
                except:
                    yafuoku_name_a = ('◆取得失敗◆')
                finally:
                    print('')
                    print('========== ヤフオク商品', yafuoku_num_current, '点目 /', len(yafuoku_item_list), '点中 ==========')
                    yafuoku_num_current += 1
                    print('Amazon商品名：', amazon_name)
                    print('ヤフオク商品名：', yafuoku_name_a)

                if 'ジャンク' in yafuoku_name_a or '訳あり' in yafuoku_name_a:
                    print('◆◆◆ ジャンク品/訳あり品のためスキップしました ◆◆◆')
                    continue

                # ヤフオク現在価格
                try:
                    yafuoku_price_now_a = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
                except:
                    yafuoku_price_now_a = ('◆取得失敗◆')
                finally:
                    print('ヤフオク現在価格：', yafuoku_price_now_a)

                # 現在価格での利益額計算
                profit_now_a = amazon_price - yafuoku_price_now_a
                print('現在価格利益：', profit_now_a, '円　◆ ヤフオク送料・Amazon出品手数料別 ◆')

                # 原罪価格での利益率計算
                profit_rate_now_a = profit_now_a / yafuoku_price_now_a
                print('現在価格利益率：', round(profit_rate_now_a * 100), '%　◆ ヤフオク送料・Amazon出品手数料別 ◆')

                # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
                # if profit_now < 0: # 利益額での判定
                    # print('◆◆◆ 想定利益がマイナスのため情報取得をスキップしました ◆◆◆')
                    # continue
                if profit_rate_now_a < 0: # 利益率での判定
                    print('◆◆◆ 想定利益率がマイナスのため情報取得をスキップしました ◆◆◆')
                    continue

                # ヤフオク商品ページを開いてアクティブにする
                yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)
                time.sleep(1)
                handle_array = browser.window_handles
                browser.switch_to.window(handle_array[-1])
                yafuoku_window = browser.current_window_handle

                print('')
                print('■ Amazonとヤフオクの商品・色は一致していますか？ y / n / s / x')
                print('■ 　y：一致している　※ 匿名配送/低評価は自動スキップします')
                print('■ 　n：一致していない（このヤフオク商品をスキップする）')
                print('■ 　s：このAmazon商品はスキップする')
                print('■ 　x：プログラムを終了する')
                while True:
                    ans = input()
                    if ans == 'y' or ans == 'n' or ans == 's' or ans == 'x':
                        break
                    else:
                        print('y か n か s か x を入力してください')

                if ans == 'y':
                    # ヤフオク商品名
                    yafuoku_name = browser.find_element(By.XPATH, '//h1').text
                    # print('ヤフオク商品名：', yafuoku_name)

                    # 出品者の評価
                    seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
                    # print('評価：', seller_rating)

                    # 評価を数値型に変換し、97％未満なら情報取得ををスキップする
                    seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))
                    if seller_rating_int < 97:
                        print('◆◆◆ 評価が低いため本商品の情報収集を終了します ◆◆◆')
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        # このヤフオク商品の情報取得をスキップする
                        break

                    # 匿名配送情報
                    try:
                        # 匿名配送希望の場合は処理をスキップする
                        privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
                        print('◆◆◆ 匿名配送のため本商品の情報取得をスキップします ◆◆◆')
                        # ヤフオクページを閉じる
                        browser.close()
                        # Amazonページをアクティブにする
                        browser.switch_to.window(amazon_window)
                        # このヤフオク商品の情報取得をスキップする
                        break
                    except:
                        privacy = '該当なし'
                        # print('匿名配送：', privacy)

                    # ヤフオク商品ページ（アクティブウィンドウ）の URL
                    yafuoku_url = browser.current_url
                    # print('ヤフオク商品URL：')
                    # print(yafuoku_url) # 表示画面から取ればいいので出力しない

                    # 評価数
                    seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
                    print('出品者評価数：', seller_rating_num)

                    # 商品の状態
                    state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
                    print('商品状態：', state)

                    # 入札数
                    bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
                    # print('入札数：', bidnum)

                    # ヤフオク現在価格
                    try:
                        yafuoku_price_now = browser.find_element(By.XPATH, '//dt[contains(text(), "現在")]/following-sibling::dd').text.replace(' ', '')
                    except:
                        yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '')
                    finally:
                        print('ヤフオク現在価格：', yafuoku_price_now)

                    # 利益計算のために int 型に変換
                    if '税込' in yafuoku_price_now: # 税抜き・税込み併記の場合
                        yafuoku_price_now_int = int(yafuoku_price_now.replace(',', '').replace('税込', '／').replace('円）', '').split('／')[1])
                    else: # 税込み表示の場合
                        yafuoku_price_now_int = int(yafuoku_price_now.replace(',', '').split('円')[0])

                    # ヤフオク即決価格
                    try:
                        # yafuoku_price_bin = browser.find_element(By.XPATH, '//dd[@class="Price__value Price__value--buyNow"]').text.replace(' ', '')
                        yafuoku_price_bin = browser.find_element(By.XPATH, '//dt[contains(text(), "即決")]/following-sibling::dd').text.replace(' ', '')

                        # ResaleTrap での入作価格入力のために int 型に変換（現在価格と同ロジック）
                        if '税込' in yafuoku_price_bin: # 税抜き・税込み併記の場合
                            yafuoku_price_bin_int = int(yafuoku_price_bin.replace(',', '').replace('税込', '／').replace('円）', '').split('／')[1])
                        else: # 税込み表示の場合
                            yafuoku_price_bin_int = int(yafuoku_price_bin.replace(',', '').split('円')[0])
                    except:
                        yafuoku_price_bin = '◆ なし ◆'
                        yafuoku_price_bin_int = '◆ なし ◆'
                    finally:
                        print('ヤフオク即決価格：', yafuoku_price_bin)

                    # 発送先を青森県に設定
                    # 「配送方法一覧」ポップアップを開く
                    browser.find_element(By.XPATH, '//a[@data-modal-name="postage"]').send_keys(Keys.ENTER)
                    browser.implicitly_wait(1)
                    # 配送先から「青森県」を選ぶ
                    browser.find_element(By.XPATH, '//option[contains(text(), "青森県")]').click()
                    browser.implicitly_wait(1)
                    # 「✕」をクリックしてポップアップを閉じる
                    browser.find_element(By.XPATH, '//div[@class="BidModal__box BidModal__box--postage js-modal-box"]//a[contains(text(), "閉じる")]').send_keys(Keys.ENTER)

                    # 配送方法
                    try:
                        shipping = browser.find_element(By.XPATH, '//dd[@class="Price__postageDetailData"]/span').text
                    except:
                        shipping = '◆ 情報なし ◆'
                    finally:
                        print('配送方法：', shipping)

                    # 送料負担
                    try:
                        postage_due = browser.find_element(By.XPATH, '//dt[contains(text(), "送料負担")]/following-sibling::dd').text
                    except:
                        postage_due = '◆ 情報なし ◆'
                    # finally:
                    #     print('送料負担：', postage_due)

                    # 送料（「配送方法一覧」ポップアップ内）
                    try:
                        postage = browser.find_element(By.XPATH, '//div[@class="BidModal__postagePrice"]').text
                        # postage = browser.find_element(By.XPATH, '//div[@class="u-paddingR60 Price__postageValue Price__postageValue--free"]').text
                    except:
                        postage =  '◆ 情報なし ◆'
                    finally:
                        print('送料：', postage)

                    if postage == '無料':
                        postage_int = 0
                    elif '円（税込）' in postage:
                        postage_int = int(postage.replace(',', '').replace('円（税込）', ''))
                    else:
                        postage_int = '◆ 送料未定 ◆'
                    # print('送料(int)：', postage_int)

                    # 青森県を選んだ状態の送料
                    try:
                        postage_area = browser.find_element(By.XPATH, '//div[@class="u-paddingR60 Price__postageValue"]').text
                    except:
                        postage_area = '◆ 情報なし ◆'
                    finally:
                        print('発送先＆送料：', postage_area)

                    # 出品地域
                    try:
                        postage_from = browser.find_elements(By.XPATH, '//dd[@class="Price__postageDetailData"]')[1].text
                    except:
                        postage_from =  '◆ 情報なし ◆'
                    finally:
                        print('出品地域：', postage_from)

                    # 発送開始日
                    try:
                        send_date = browser.find_element(By.XPATH, '//dt[contains(text(), "発送開始")]/following-sibling::dd').text
                    except:
                        send_date = '◆ 情報なし ◆'
                    finally:
                        print('発送開始：', send_date)

                    # オークション残り日数
                    auc_timeleft = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[1].text
                    # print('残り日数：', auc_timeleft)

                    # 終了予定日時
                    auc_enddatetime = browser.find_element(By.XPATH, '//th[contains(text(), "終了日時")]/following-sibling::td').text
                    # print('終了予定：', auc_enddatetime)

                    # オークション残り日時詳細計算
                    auc_enddate = auc_enddatetime.split('（')[0]
                    auc_endtime = auc_enddatetime.split('）')[1]
                    auc_end = auc_enddate + ' ' + auc_endtime
                    auc_enddatetime = datetime.datetime.strptime(auc_end, '%Y.%m.%d %H:%M')
                    # print('終了予定(datetime型)：', auc_enddatetime)
                    dt_now = datetime.datetime.now()
                    # print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))
                    auc_time_left = auc_enddatetime - dt_now
                    # print('残り時間(詳細計算)：', auc_time_left)

                    # ヤフオク送料も加味した現在価格での利益額/利益率　※ Amazon 出品送料は ResaleTrap で判定
                    if postage_int == '◆ 送料未定 ◆':
                        profit_now_final = amazon_price - yafuoku_price_now_int
                        print('現在価格利益：', profit_now_final, '円　◆ ヤフオク送料・Amazon出品手数料別 ◆')

                        profit_rate_now_final = round(profit_now_final / yafuoku_price_now_int * 100)
                        print('現在価格利益率：', profit_rate_now_final, '%　◆ ヤフオク送料・Amazon出品手数料別 ◆')
                    else:
                        profit_now_final = amazon_price - yafuoku_price_now_int - postage_int
                        print('現在価格利益(送料込)：', profit_now_final, '円　◆ Amazon出品手数料別 ◆')

                        profit_rate_now_final = round(profit_now_final / yafuoku_price_now_int * 100)
                        print('現在価格利益率(送料込)：', profit_rate_now_final, '%　◆ Amazon出品手数料別 ◆')

                    # ～～～～～ 出品可否判定 ココカラ ～～～～～
                    print('')
                    print('----- 商品説明内の文言判定 -----')

                    # 商品説明
                    yafuoku_info = browser.find_element(By.XPATH, '//div[@class="ProductExplanation__commentArea"]').text
                    # print('商品説明：', yafuoku_info)

                    # 送料関係
                    print('青森/東北送料：', re.findall('.*青森.*', yafuoku_info), re.findall('.*東北.*', yafuoku_info))
                    print('')

                    # 直接引き取り判定
                    # print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))
                    print('直接引取：', re.findall('.*直接.*引.*', yafuoku_info))
                    print('手渡し/店頭渡し', re.findall('.*渡し.*', yafuoku_info))

                    # 着払い判定
                    print('着払：', re.findall('.*着払.*', yafuoku_info))

                    # 配送業者、営業所止め判定
                    print('ヤマト：', re.findall('.*ヤマト.*', yafuoku_info), re.findall('.*らくらく.*', yafuoku_info))
                    print('佐川', re.findall('.*佐川.*', yafuoku_info))
                    print('日本郵便', re.findall('.*日本郵便.*', yafuoku_info), re.findall('.*ゆう.*', yafuoku_info))
                    print('営業所止め', re.findall('.*止め.*', yafuoku_info))

                    # 動作状況、付属品状況
                    print('動作確認：', re.findall('.*動作.*', yafuoku_info))
                    print('訳あり：', re.findall('.*訳.*', yafuoku_info))
                    print('ジャンク：', re.findall('.*ジャンク.*', yafuoku_info))
                    print('付属品：', re.findall('.*付属.*', yafuoku_info), re.findall('.*写真.*', yafuoku_info))
                    # ～～～～～ 出品可否判定 ココマデ ～～～～～

                    # ～～～～～ 出品判定 ココカラ ～～～～～
                    print('')
                    print('■ この商品を出品候補にしますか？ y / n')
                    while True:
                        pre_sell_flag = input()
                        if pre_sell_flag == 'y' or pre_sell_flag == 'n':
                            break
                        else:
                            print('y か n を入力してください')

                    if pre_sell_flag == 'y':
                        print('出品画面に進みます')
                        # 「監視対象に追加する」をクリックする。ResaleTrap のページが開かれる
                        browser.find_element(By.XPATH, '//a[contains(text(), "監視対象に追加する")]').send_keys(Keys.ENTER)

                        # ResaleTrap ページをアクティブにする
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        resaletrap_window = browser.current_window_handle
                        time.sleep(3)
                        
                        # 即決価格があれば入力
                        # if yafuoku_price_bin != '◆ なし ◆':
                        #     browser.find_elements(By.XPATH, '//input')[0].send_keys(Keys.CONTROL,"a")
                        #     browser.find_elements(By.XPATH, '//input')[0].send_keys(yafuoku_price_bin_int)
                            
                        # ResaleTrap 上の即決価格を予想落札価格に転記
                        rt_price_bin = browser.find_elements(By.XPATH, '//table[@class="table"]//td')[3].text.replace(',', '').replace('円', '')
                        if rt_price_bin != '-':
                            browser.find_elements(By.XPATH, '//input')[0].send_keys(Keys.CONTROL,"a")
                            browser.find_elements(By.XPATH, '//input')[0].send_keys(rt_price_bin)                            

                        # 送料入力
                        if postage_int != '◆ 送料未定 ◆':
                            browser.find_elements(By.XPATH, '//input')[1].send_keys(Keys.CONTROL,"a")
                            browser.find_elements(By.XPATH, '//input')[1].send_keys(postage_int)

                        # 商品説明　テンプレート選択
                        browser.find_element(By.XPATH, '//option[@value="6483"]').click()

                        print('')
                        print('■ この商品を出品しますか？ y / n')
                        print('■ 　出品する → 各項目を入力して出品し、ページは閉じずに y と入力してください')
                        print('■　　　　　　　※ 即決価格がない場合、予想落札価格は同商品のアクティブなオークションの現在価格の最高額と過去の落札価格の最高額の高い方')
                        print('■ 　やめる　 → n と入力してください')
                        while True:
                            sell_done_flag = input()
                            if sell_done_flag == 'y' or sell_done_flag == 'n':
                                break
                            else:
                                print('y か n を入力してください')

                        # 取得した情報をリストに追加して CSV ファイルに出力する。ファイル名(csv_file_name)はプログラム冒頭で定義済。情報を追加しながら上書き
                        if sell_done_flag == 'y':
                            data = {
                                'Amazon商品名': amazon_name,
                                'Amazon価格[円]': amazon_price,
                                'Amazon商品URL': amazon_url,
                                'ヤフオク商品名': yafuoku_name,
                                'ヤフオク現在価格[円]': yafuoku_price_now_int,
                                'ヤフオク即決価格[円]': yafuoku_price_bin,
                                '送料': postage_int,
                                '発送先＆送料': postage_area,
                                '匿名配送': privacy,
                                '状態': state,
                                '評価[%]': seller_rating_int,
                                '評価数': seller_rating_num,
                                'ヤフオク商品URL': yafuoku_url,
                                '現在価格利益(送料込)[円]': profit_now_final,
                                '現在価格利益率(送料込)[%]': profit_rate_now_final,
                                '残り日数': auc_timeleft,
                                '発送開始': send_date,
                                '情報取得日時': datetime.datetime.now().strftime('%Y/%m/%d %H:%M'),
                                }
                            data_list.append(data)
                            pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')
                            print('★★★★★ 取得情報をファイルに出力しました ★★★★★')
                        elif sell_done_flag == 'n':
                            pass

                        # ResaleTrap ページを閉じる
                        browser.close()
                        # ヤフオク商品ページを閉じる
                        browser.switch_to.window(yafuoku_window)
                        browser.close()
                        # Amazon ページをアクティブにする
                        browser.switch_to.window(amazon_window)
                    elif pre_sell_flag == 'n':
                        print('◆◆◆ この商品の出品をスキップしました ◆◆◆')
                        # ヤフオク商品ページを閉じる
                        browser.close()
                        # Amazon ページをアクティブにする
                        browser.switch_to.window(amazon_window)
                    # ～～～～～ 出品判定 ココマデ ～～～～～
                elif ans == 'n':
                    print('◆◆◆ この商品の情報取得をスキップしました ◆◆◆')
                    browser.close()
                    browser.switch_to.window(amazon_window)
                elif ans == 's':
                    print('◆◆◆ このAmazon商品に関する残りのヤフオク商品情報取得をスキップしました ◆◆◆')
                    browser.close()
                    browser.switch_to.window(amazon_window)
                    break
                elif ans == 'x':
                    print('プログラムを終了します')
                    # ヤフオクページを閉じる
                    browser.close()
                    # Amazon ページを閉じる
                    browser.switch_to.window(amazon_window)
                    browser.close()
                    # プログラムを終了する
                    sys.exit()
                # print('===== 次のヤフオク商品に移ります =====')
            # print('===== 次の Amazon 商品に移ります =====')
        try:
            url = browser.find_element(By.XPATH, '//a[contains(text(), "次へ")]').get_attribute('href')
            print('')
            print('■ 次のページへ移動します。Enter を押してください')
            input()
        except:
            print('')
            print('■ このページが最終ページのため情報取得を終了します。Enter でプログラムを終了します')
            input()
            sys.exit()

if __name__ == '__main__':
    main(url)

In [5]:
# 2023/06/18：Amazon商品名に「Amazon限定」関連の文言がある場合はスキップ
# 2023/06/23：Amazon商品名に「アイリスオーヤマ」がある場合はスキップ
# 2023/06/23：ヤフオクぉく商品名に「ジャンク」とあったらスキップ
# 2023/06/24：全3つのwhileループ判定を効率化してインデントを簡素化
# 2023/06/24：特定のAmazon商品に対するヤフオク商品の情報取得を一括スキップする方法（多重ループからのbreak：選択肢「S」）を追加
# 2023/06/24：ヤフオクぉく商品名に「訳あり」とあったらスキップ
# 2023/06/25：スポンサー商品をスキップ
# 2023/06/26：最終ページかどうかの判定を追加
# 2023/06/28：ResaleTrap ページで即決価格入力、送料入力、商品説明テンプレート選択を自動化
# 2023/06/28：次ページへの移動を簡素化
# 2023/06/30：ヤフオクの送料構造が「詳細」ポップアップから「配送方法一覧」に変わったことに対応
# 2023/06/30：現在価格と即決価格の取り方 XPATH を改良して即決価格だけの場合にも正しく取れるようにした

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# このプログラムで取得する Amazon・ヤフオク情報を格納するためのリスト準備
data_list = []

# 格納した情報の出力先ファイル名
output_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'

# 調査対象のAmazon商品ページを開く（固定入力パターン）
# url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771&dc&fs=true&ds=v1%3A%2F4aS563Nkvk0UGsPleVbnbegcIvY1CBz%2B04FQKejLmc&qid=1686834426&ref=sr_ex_n_1' # キッチン家電
# url = 'https://www.amazon.co.jp/s?k=%E3%82%A2%E3%83%9E%E3%82%BE%E3%83%B3+%E9%99%90%E5%AE%9A&__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&crid=3HVT9UVZAI6YQ&sprefix=%E3%82%A2%E3%83%9E%E3%82%BE%E3%83%B3+%E9%99%90%E5%AE%9A%2Caps%2C196&ref=nb_sb_noss_1' # アマゾン限定品

# 調査対象のAmazon商品ページを開く（手動入力パターン）
print('■ 調査対象の Amazon URL を入力してください')
url = input()

def main(url):
    # 2023/06/18：次のページに移動する処理を追加
    while True:
        browser.get(url)
        browser.implicitly_wait(3)
        # Amazon のウィンドウハンドラの保持
        amazon_window = browser.current_window_handle

        print('※ ResaleTrapの検索結果の整合性が低い場合は別のAmazonページに切り替えることをオススメします')
        print('')
        
        # 途中でエラーになった場合に備えて任意の位置(商品)から再開できるようする
        amazon_num_current = 1 # Amazon商品の現在番号
        print('■ ResaleTrapにログインしていなければログインしてください')
        print('staff2@polyphony.tokyo')
        print('Muzaiko20230606')
        print('')
        print('■ 何番目のAmazon商品から情報取得しますか？ 1～29　※スポンサー商品/Amazon限定品は自動スキップします')
        while True:
            amazon_num_temp = input() # 情報取得開始番号
            if re.fullmatch('[0-2]?[0-9]', amazon_num_temp):
                amazon_num_start = int(amazon_num_temp)
                break
            else:
                print('1～29の半角数字を入力してください')

        # Amazon商品＋ヤフオク商品群の基点要素を取得する
        amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

        # Amazon＋ヤフオク商品群の親要素内の情報を取得する
        for amazon_elem in amazon_yafuoku_item_list:

            # 途中でエラーになった場合に備えた仕組み。Amazon商品の情報取得を上で指定した番号までスキップする
            if amazon_num_current < amazon_num_start:
                amazon_num_current += 1
                continue

            print('')
            print('■■■■■■■■■■■■■■■ Amazon商品', amazon_num_current, '点目 ■■■■■■■■■■■■■■■')
            amazon_num_current += 1

            # Amazon商品名
            amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
            print(amazon_name)

            # Amazon限定商品はスキップする
            if '【Amazon限定ブランド】' in amazon_name \
              or '[Amazon限定ブランド]' in amazon_name \
              or '【Amazon 限定ブランド】' in amazon_name \
              or '【Amazon.co.jp限定】' in amazon_name \
              or '【Amazon.co.jp 限定】' in amazon_name \
              or '[Amazonブランド]' in amazon_name \
              or '【アマゾン限定】' in amazon_name:
                print('◆◆◆ Amazon限定品のためスキップしました ◆◆◆')
                continue
            # アイリスオーヤマ製品は注意 or スキップする
            elif 'アイリスオーヤマ' in amazon_name:
                print('◆◆◆ アイリスオーヤマ製品です。テレビの場合はスキップしてください ◆◆◆')

            # スポンサー商品はスキップする
            try:
                if amazon_elem.find_element(By.XPATH, './/span[contains(text(), "スポンサー")]').text == 'スポンサー':
                    print('◆◆◆ スポンサー商品のためスキップしました ◆◆◆')
                    continue
            except:
                pass

            # Amazon購入実績
            # try:
            #     amazon_sold_num = amazon_elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
            # except:
            #     amazon_sold_num = '◆Amazon購入実績情報なし◆'
            # finally:
                # print('Amazon購入実績：', amazon_sold_num)

            # Amazon価格
            try:
                amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
                print(amazon_price)
            except:
                amazon_price = '◆Amazon価格情報なし◆'
                print(amazon_price)
                continue

            # Amazon商品URL
            amazon_url = amazon_elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
            print(amazon_url)

            # ヤフオク商品群の基点要素を取得する
            yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')
            yafuoku_num_current = 1 # 同一Amazon商品内のヤフオク商品の現在番号

            # ヤフオク商品群の親要素内の情報を取得する
            for yafuoku_elem in yafuoku_item_list:
                # ヤフオク商品名
                try:
                    yafuoku_name_a = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
                except:
                    yafuoku_name_a = ('◆取得失敗◆')
                finally:
                    print('')
                    print('========== ヤフオク商品', yafuoku_num_current, '点目 /', len(yafuoku_item_list), '点中 ==========')
                    yafuoku_num_current += 1
                    print('Amazon商品名：', amazon_name)
                    print('ヤフオク商品名：', yafuoku_name_a)

                if 'ジャンク' in yafuoku_name_a or '訳あり' in yafuoku_name_a:
                    print('◆◆◆ ジャンク品/訳あり品のためスキップしました ◆◆◆')
                    continue

                # ヤフオク現在価格
                try:
                    yafuoku_price_now_a = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
                except:
                    yafuoku_price_now_a = ('◆取得失敗◆')
                finally:
                    print('ヤフオク現在価格：', yafuoku_price_now_a)

                # 現在価格での利益額計算
                profit_now_a = amazon_price - yafuoku_price_now_a
                print('現在価格利益：', profit_now_a, '円　◆ ヤフオク送料・Amazon出品手数料別 ◆')

                # 原罪価格での利益率計算
                profit_rate_now_a = profit_now_a / yafuoku_price_now_a
                print('現在価格利益率：', round(profit_rate_now_a * 100), '%　◆ ヤフオク送料・Amazon出品手数料別 ◆')

                # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
                # if profit_now < 0: # 利益額での判定
                    # print('◆◆◆ 想定利益がマイナスのため情報取得をスキップしました ◆◆◆')
                    # continue
                if profit_rate_now_a < 0: # 利益率での判定
                    print('◆◆◆ 想定利益率がマイナスのため情報取得をスキップしました ◆◆◆')
                    continue

                # ヤフオク商品ページを開いてアクティブにする
                yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)
                time.sleep(1)
                handle_array = browser.window_handles
                browser.switch_to.window(handle_array[-1])
                yafuoku_window = browser.current_window_handle

                print('')
                print('■ Amazonとヤフオクの商品・色は一致していますか？ y / n / s / x')
                print('■ 　y：一致している　※ 匿名配送/低評価は自動スキップします')
                print('■ 　n：一致していない（このヤフオク商品をスキップする）')
                print('■ 　s：このAmazon商品はスキップする')
                print('■ 　x：プログラムを終了する')
                while True:
                    ans = input()
                    if ans == 'y' or ans == 'n' or ans == 's' or ans == 'x':
                        break
                    else:
                        print('y か n か s か x を入力してください')

                if ans == 'y':
                    # ヤフオク商品名
                    yafuoku_name = browser.find_element(By.XPATH, '//h1').text
                    # print('ヤフオク商品名：', yafuoku_name)

                    # 出品者の評価
                    seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
                    # print('評価：', seller_rating)

                    # 評価を数値型に変換し、97％未満なら情報取得ををスキップする
                    seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))
                    if seller_rating_int < 97:
                        print('◆◆◆ 評価が低いため本商品の情報収集を終了します ◆◆◆')
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        # このヤフオク商品の情報取得をスキップする
                        break

                    # 匿名配送情報
                    try:
                        # 匿名配送希望の場合は処理をスキップする
                        privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
                        print('◆◆◆ 匿名配送のため本商品の情報取得をスキップします ◆◆◆')
                        # ヤフオクページを閉じる
                        browser.close()
                        # Amazonページをアクティブにする
                        browser.switch_to.window(amazon_window)
                        # このヤフオク商品の情報取得をスキップする
                        break
                    except:
                        privacy = '該当なし'
                        # print('匿名配送：', privacy)

                    # ヤフオク商品ページ（アクティブウィンドウ）の URL
                    yafuoku_url = browser.current_url
                    # print('ヤフオク商品URL：')
                    # print(yafuoku_url) # 表示画面から取ればいいので出力しない

                    # 評価数
                    seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
                    # print('出品者評価数：', seller_rating_num)

                    # 商品の状態
                    state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
                    # print('商品状態：', state)

                    # 入札数
                    bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
                    # print('入札数：', bidnum)

                    # ヤフオク現在価格
                    try:
                        yafuoku_price_now = browser.find_element(By.XPATH, '//dt[contains(text(), "現在")]/following-sibling::dd').text.replace(' ', '')
                    except:
                        yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '')
                    # finally:
                    #     print('ヤフオク現在価格：', yafuoku_price_now)

                    # 利益計算のために int 型に変換
                    if '税込' in yafuoku_price_now: # 税抜き・税込み併記の場合
                        yafuoku_price_now_int = int(yafuoku_price_now.replace(',', '').replace('税込', '／').replace('円）', '').split('／')[1])
                    else: # 税込み表示の場合
                        yafuoku_price_now_int = int(yafuoku_price_now.replace(',', '').split('円')[0])

                    # ヤフオク即決価格
                    try:
                        # yafuoku_price_bin = browser.find_element(By.XPATH, '//dd[@class="Price__value Price__value--buyNow"]').text.replace(' ', '')
                        yafuoku_price_bin = browser.find_element(By.XPATH, '//dt[contains(text(), "即決")]/following-sibling::dd').text.replace(' ', '')

                        # ResaleTrap での入作価格入力のために int 型に変換（現在価格と同ロジック）
                        if '税込' in yafuoku_price_bin: # 税抜き・税込み併記の場合
                            yafuoku_price_bin_int = int(yafuoku_price_bin.replace(',', '').replace('税込', '／').replace('円）', '').split('／')[1])
                        else: # 税込み表示の場合
                            yafuoku_price_bin_int = int(yafuoku_price_bin.replace(',', '').split('円')[0])
                    except:
                        yafuoku_price_bin = '◆ なし ◆'
                        yafuoku_price_bin_int = '◆ なし ◆'
                    # finally:
                    #     print('ヤフオク即決価格：', yafuoku_price_bin)

                    # 発送先を青森県に設定
                    # 「配送方法一覧」ポップアップを開く
                    browser.find_element(By.XPATH, '//a[@data-modal-name="postage"]').send_keys(Keys.ENTER)
                    browser.implicitly_wait(1)
                    # 配送先から「青森県」を選ぶ
                    browser.find_element(By.XPATH, '//option[contains(text(), "青森県")]').click()
                    browser.implicitly_wait(1)
                    # 「✕」をクリックしてポップアップを閉じる
                    browser.find_element(By.XPATH, '//div[@class="BidModal__box BidModal__box--postage js-modal-box"]//a[contains(text(), "閉じる")]').send_keys(Keys.ENTER)

                    # 配送方法
                    try:
                        shipping = browser.find_element(By.XPATH, '//dd[@class="Price__postageDetailData"]/span').text
                    except:
                        shipping = '◆ 情報なし ◆'
                    finally:
                        print('配送方法：', shipping)

                    # 送料負担
                    try:
                        postage_due = browser.find_element(By.XPATH, '//dt[contains(text(), "送料負担")]/following-sibling::dd').text
                    except:
                        postage_due = '◆ 情報なし ◆'
                    # finally:
                    #     print('送料負担：', postage_due)

                    # 送料（「配送方法一覧」ポップアップ内）
                    try:
                        postage = browser.find_element(By.XPATH, '//div[@class="BidModal__postagePrice"]').text
                        # postage = browser.find_element(By.XPATH, '//div[@class="u-paddingR60 Price__postageValue Price__postageValue--free"]').text
                    except:
                        postage =  '◆ 情報なし ◆'
                    finally:
                        print('送料：', postage)

                    if postage == '無料':
                        postage_int = 0
                    elif '円（税込）' in postage:
                        postage_int = int(postage.replace(',', '').replace('円（税込）', ''))
                    else:
                        postage_int = '◆ 送料未定 ◆'
                    # print('送料(int)：', postage_int)

                    # 青森県を選んだ状態の送料
                    try:
                        # postage_area = browser.find_element(By.XPATH, '//div[@class="u-paddingR60 Price__postageValue"]').text # パターン１
                        postage_area = browser.find_element(By.XPATH, '//dd[@class="Price__postageData"]/div').text # パターン２
                    except:
                        postage_area = '◆ 情報なし ◆'
                    finally:
                        print('発送先＆送料：', postage_area)

                    # 出品地域
                    try:
                        postage_from = browser.find_elements(By.XPATH, '//dd[@class="Price__postageDetailData"]')[1].text
                    except:
                        postage_from =  '◆ 情報なし ◆'
                    # finally:
                    #     print('出品地域：', postage_from)

                    # 発送開始日
                    try:
                        send_date = browser.find_element(By.XPATH, '//dt[contains(text(), "発送開始")]/following-sibling::dd').text
                    except:
                        send_date = '◆ 情報なし ◆'
                    # finally:
                    #     print('発送開始：', send_date)

                    # オークション残り日数
                    auc_timeleft = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[1].text
                    # print('残り日数：', auc_timeleft)

                    # 終了予定日時
                    auc_enddatetime = browser.find_element(By.XPATH, '//th[contains(text(), "終了日時")]/following-sibling::td').text
                    # print('終了予定：', auc_enddatetime)

                    # オークション残り日時詳細計算
                    auc_enddate = auc_enddatetime.split('（')[0]
                    auc_endtime = auc_enddatetime.split('）')[1]
                    auc_end = auc_enddate + ' ' + auc_endtime
                    auc_enddatetime = datetime.datetime.strptime(auc_end, '%Y.%m.%d %H:%M')
                    # print('終了予定(datetime型)：', auc_enddatetime)
                    dt_now = datetime.datetime.now()
                    # print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))
                    auc_time_left = auc_enddatetime - dt_now
                    # print('残り時間(詳細計算)：', auc_time_left)

                    # ヤフオク送料も加味した現在価格での利益額/利益率　※ Amazon 出品送料は ResaleTrap で判定
                    if postage_int == '◆ 送料未定 ◆':
                        profit_now_final = amazon_price - yafuoku_price_now_int
                        # print('現在価格利益：', profit_now_final, '円　◆ ヤフオク送料・Amazon出品手数料別 ◆')

                        profit_rate_now_final = round(profit_now_final / yafuoku_price_now_int * 100)
                        # print('現在価格利益率：', profit_rate_now_final, '%　◆ ヤフオク送料・Amazon出品手数料別 ◆')
                    else:
                        profit_now_final = amazon_price - yafuoku_price_now_int - postage_int
                        # print('現在価格利益(送料込)：', profit_now_final, '円　◆ Amazon出品手数料別 ◆')

                        profit_rate_now_final = round(profit_now_final / yafuoku_price_now_int * 100)
                        # print('現在価格利益率(送料込)：', profit_rate_now_final, '%　◆ Amazon出品手数料別 ◆')

                    # ～～～～～ 出品可否判定 ココカラ ～～～～～
                    print('')
                    print('----- 商品説明内の文言判定 -----')

                    # 商品説明
                    yafuoku_info = browser.find_element(By.XPATH, '//div[@class="ProductExplanation__commentArea"]').text
                    # print('商品説明：', yafuoku_info)

                    # 送料関係
                    print('青森/東北送料：', re.findall('.*青森.*', yafuoku_info), re.findall('.*東北.*', yafuoku_info))
                    print('')

                    # 直接引き取り判定
                    # print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))
                    print('直接引取：', re.findall('.*直接.*引.*', yafuoku_info))
                    print('手渡し/店頭渡し', re.findall('.*渡し.*', yafuoku_info))

                    # 着払い判定
                    print('着払：', re.findall('.*着払.*', yafuoku_info))

                    # 配送業者、営業所止め判定
                    print('ヤマト：', re.findall('.*ヤマト.*', yafuoku_info), re.findall('.*らくらく.*', yafuoku_info))
                    print('佐川', re.findall('.*佐川.*', yafuoku_info))
                    print('日本郵便', re.findall('.*日本郵便.*', yafuoku_info), re.findall('.*ゆう.*', yafuoku_info))
                    print('営業所止め', re.findall('.*止め.*', yafuoku_info))

                    # 動作状況、付属品状況
                    print('動作確認：', re.findall('.*動作.*', yafuoku_info))
                    print('訳あり：', re.findall('.*訳.*', yafuoku_info))
                    print('ジャンク：', re.findall('.*ジャンク.*', yafuoku_info))
                    print('付属品：', re.findall('.*付属.*', yafuoku_info), re.findall('.*写真.*', yafuoku_info))
                    # ～～～～～ 出品可否判定 ココマデ ～～～～～

                    # ～～～～～ 出品判定 ココカラ ～～～～～
                    print('')
                    print('■ この商品を出品候補にしますか？ y / n')
                    while True:
                        pre_sell_flag = input()
                        if pre_sell_flag == 'y' or pre_sell_flag == 'n':
                            break
                        else:
                            print('y か n を入力してください')

                    if pre_sell_flag == 'y':
                        print('出品画面に進みます')
                        # 「監視対象に追加する」をクリックする。ResaleTrap のページが開かれる
                        browser.find_element(By.XPATH, '//a[contains(text(), "監視対象に追加する")]').send_keys(Keys.ENTER)

                        # ResaleTrap ページをアクティブにする
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        resaletrap_window = browser.current_window_handle
                        time.sleep(3)
                        
                        # 即決価格があれば入力
                        # if yafuoku_price_bin != '◆ なし ◆':
                        #     browser.find_elements(By.XPATH, '//input')[0].send_keys(Keys.CONTROL,"a")
                        #     browser.find_elements(By.XPATH, '//input')[0].send_keys(yafuoku_price_bin_int)
                            
                        # ResaleTrap 上の即決価格を予想落札価格に転記
                        rt_price_bin = browser.find_elements(By.XPATH, '//table[@class="table"]//td')[3].text.replace(',', '').replace('円', '')
                        if rt_price_bin != '-':
                            browser.find_elements(By.XPATH, '//input')[0].send_keys(Keys.CONTROL,"a")
                            browser.find_elements(By.XPATH, '//input')[0].send_keys(rt_price_bin)                            

                        # 送料入力
                        if postage_int != '◆ 送料未定 ◆':
                            browser.find_elements(By.XPATH, '//input')[1].send_keys(Keys.CONTROL,"a")
                            browser.find_elements(By.XPATH, '//input')[1].send_keys(postage_int)

                        # 商品説明　テンプレート選択
                        browser.find_element(By.XPATH, '//option[@value="6483"]').click()

                        print('')
                        print('■ この商品を出品しますか？ y / n')
                        print('■ 　出品する → 各項目を入力して出品し、ページは閉じずに y と入力してください')
                        print('■　　　　　　　※ 即決価格がない場合、予想落札価格は同商品のアクティブなオークションの現在価格の最高額と過去の落札価格の最高額の高い方')
                        print('■ 　やめる　 → n と入力してください')
                        while True:
                            sell_done_flag = input()
                            if sell_done_flag == 'y' or sell_done_flag == 'n':
                                break
                            else:
                                print('y か n を入力してください')

                        # 取得した情報をリストに追加して CSV ファイルに出力する。ファイル名(csv_file_name)はプログラム冒頭で定義済。情報を追加しながら上書き
                        if sell_done_flag == 'y':
                            data = {
                                'Amazon商品名': amazon_name,
                                'Amazon価格[円]': amazon_price,
                                'Amazon商品URL': amazon_url,
                                'ヤフオク商品名': yafuoku_name,
                                'ヤフオク現在価格[円]': yafuoku_price_now_int,
                                'ヤフオク即決価格[円]': yafuoku_price_bin,
                                '送料': postage_int,
                                '発送先＆送料': postage_area,
                                '匿名配送': privacy,
                                '状態': state,
                                '評価[%]': seller_rating_int,
                                '評価数': seller_rating_num,
                                'ヤフオク商品URL': yafuoku_url,
                                '現在価格利益(送料込)[円]': profit_now_final,
                                '現在価格利益率(送料込)[%]': profit_rate_now_final,
                                '残り日数': auc_timeleft,
                                '発送開始': send_date,
                                '情報取得日時': datetime.datetime.now().strftime('%Y/%m/%d %H:%M'),
                                }
                            data_list.append(data)
                            pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')
                            print('★★★★★ 取得情報をファイルに出力しました ★★★★★')
                        elif sell_done_flag == 'n':
                            pass

                        # ResaleTrap ページを閉じる
                        browser.close()
                        # ヤフオク商品ページを閉じる
                        browser.switch_to.window(yafuoku_window)
                        browser.close()
                        # Amazon ページをアクティブにする
                        browser.switch_to.window(amazon_window)
                    elif pre_sell_flag == 'n':
                        print('◆◆◆ この商品の出品をスキップしました ◆◆◆')
                        # ヤフオク商品ページを閉じる
                        browser.close()
                        # Amazon ページをアクティブにする
                        browser.switch_to.window(amazon_window)
                    # ～～～～～ 出品判定 ココマデ ～～～～～
                elif ans == 'n':
                    print('◆◆◆ この商品の情報取得をスキップしました ◆◆◆')
                    browser.close()
                    browser.switch_to.window(amazon_window)
                elif ans == 's':
                    print('◆◆◆ このAmazon商品に関する残りのヤフオク商品情報取得をスキップしました ◆◆◆')
                    browser.close()
                    browser.switch_to.window(amazon_window)
                    break
                elif ans == 'x':
                    print('プログラムを終了します')
                    # ヤフオクページを閉じる
                    browser.close()
                    # Amazon ページを閉じる
                    browser.switch_to.window(amazon_window)
                    browser.close()
                    # プログラムを終了する
                    sys.exit()
                # print('===== 次のヤフオク商品に移ります =====')
            # print('===== 次の Amazon 商品に移ります =====')
        try:
            url = browser.find_element(By.XPATH, '//a[contains(text(), "次へ")]').get_attribute('href')
            print('')
            print('■ 次のページへ移動します。Enter を押してください')
            input()
        except:
            print('')
            print('■ このページが最終ページのため情報取得を終了します。Enter でプログラムを終了します')
            input()
            browser.close()
            sys.exit()

if __name__ == '__main__':
    main(url)

C:\Users\amuza\AppData\Local\Temp\ipykernel_3604\222850504.py:35: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)


■ 調査対象の Amazon URL を入力してください


 https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771%2Cn%3A4083471%2Cn%3A2276690051&dc&page=4&qid=1690383829&rnid=4083471&ref=sr_pg_4


※ ResaleTrapの検索結果の整合性が低い場合は別のAmazonページに切り替えることをオススメします

■ ResaleTrapにログインしていなければログインしてください
staff2@polyphony.tokyo
Muzaiko20230606

■ 何番目のAmazon商品から情報取得しますか？ 1～29　※スポンサー商品/Amazon限定品は自動スキップします


 1


...


■■■■■■■■■■■■■■■ Amazon商品 1 点目 ■■■■■■■■■■■■■■■
クオーレ コーヒーメーカー ブルー CU-52CD(B)
◆◆◆ スポンサー商品のためスキップしました ◆◆◆

■■■■■■■■■■■■■■■ Amazon商品 2 点目 ■■■■■■■■■■■■■■■
クオーレ コーヒーメーカー ピンク CU-52CD(P)
◆◆◆ スポンサー商品のためスキップしました ◆◆◆

■■■■■■■■■■■■■■■ Amazon商品 3 点目 ■■■■■■■■■■■■■■■
アラカルト コーヒーメーカー ライトネイビー MCD-552
◆◆◆ スポンサー商品のためスキップしました ◆◆◆

■■■■■■■■■■■■■■■ Amazon商品 4 点目 ■■■■■■■■■■■■■■■
ラッセルホブス コーヒーメーカー 5杯 ペーパーフィルター不要 タイマー ベーシックドリップ 7620JP
6800
https://www.amazon.co.jp/Russell-Hobbs-%E3%83%A9%E3%83%83%E3%82%BB%E3%83%AB%E3%83%9B%E3%83%96%E3%82%B9-7620JP-5%E3%82%AB%E3%83%83%E3%83%97%E3%82%B3%E3%83%BC%E3%83%92%E3%83%BC%E3%83%A1%E3%83%BC%E3%82%AB%E3%83%BC/dp/B07JGRG8CJ/ref=sr_1_124?qid=1690385165&rnid=4083471&s=kitchen&sr=1-124

========== ヤフオク商品 1 点目 / 7 点中 ==========
Amazon商品名： ラッセルホブス コーヒーメーカー 5杯 ペーパーフィルター不要 タイマー ベーシックドリップ 7620JP
ヤフオク商品名： ラッセルホブス コーヒーメーカー 5杯 ペーパーフィルター不要 タイマー ベーシックドリップ 7620JP
ヤフオク現在価格： 6593
現在価格利益： 207 円　◆ ヤフオク送料・Amazon出品手数料別 ◆
現在価格利益率： 3 %　◆ ヤフオク送料・Amazon出品手数料別 ◆

■ Amazonとヤフオクの商品・色は一致していますか？ y / n / 

 x


プログラムを終了します


SystemExit: 

C:\Users\amuza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
